In [1]:
class Person:

    def __call__(self):
        print("__call__ called...")


In [2]:
p = Person()
p()

__call__ called...


In [3]:
type(p)

__main__.Person

In [4]:
from functools import partial

type(partial)

type

In [5]:
def my_func(a, b, c):
    return a, b, c

In [6]:
type(my_func)

function

In [7]:
partial_func  = partial(my_func, 10, 20)
type(partial_func)

functools.partial

In [8]:
partial_func(30)

(10, 20, 30)

In [9]:
class MyPartial:

    def __init__(self, func, *args):
        self._func = func
        self._args = args

    def __call__(self, *args):
        return self._func(*self._args, *args)

In [10]:
partial_func = MyPartial(my_func, 10, 20)
partial_func(30)


(10, 20, 30)

In [11]:
callable(partial_func)

True

In [12]:
callable(MyPartial)

True

In [13]:
from collections import defaultdict

In [14]:
def default_value():
    return "N/A"

d = defaultdict(default_value)

In [15]:
d["a"]

'N/A'

In [16]:
d["b"] = 100
d["b"]

100

In [17]:
d.items()

dict_items([('a', 'N/A'), ('b', 100)])

In [18]:
miss_counter = 0

def default_value():
    global miss_counter
    miss_counter += 1
    return "N/A"

d = defaultdict(default_value)

In [19]:
d["a"] = 1

In [20]:
d["b"], miss_counter

('N/A', 1)

In [21]:
miss_counter = 0

def default_value(counter=miss_counter):
    counter += 1  # creates local variable so it won't work, integers are not mutable
    print(counter is miss_counter)
    return "N/A"

In [22]:
default_value(), default_value(), default_value(), default_value(), default_value.__dict__, miss_counter,

False
False
False
False


('N/A', 'N/A', 'N/A', 'N/A', {}, 0)

In [23]:
class DefaultValue:
    def __init__(self):
        self.counter = 0

    def __iadd__(self, other):
        if isinstance(other, int):
            self.counter += other
            return self
        raise ValueError("Can only icrement with an integer value")

    def __call__(self):
        self.counter += 1
        print(f"__call__ Counter: {self.counter}")
        

default_value_1 = DefaultValue()
default_value_2 = DefaultValue()

default_value_1.counter, default_value_1()

__call__ Counter: 1


(0, None)

In [24]:
cache_1 = defaultdict(default_value_1)
cache_2 = defaultdict(default_value_2)

In [25]:
cache_1["a"], cache_1["b"], cache_1["c"]

__call__ Counter: 2
__call__ Counter: 3
__call__ Counter: 4


(None, None, None)

In [26]:
cache_2["a"]

__call__ Counter: 1


In [27]:
class DefaultValue:
    def __init__(self, default_value):
        self.counter = 0
        self.default_value = default_value

    def __call__(self):
        self.counter += 1
        print(f"__call__ Counter: {self.counter}")
        return self.default_value
        

default_value_1 = DefaultValue(None)
default_value_2 = DefaultValue(0)

cache_1 = defaultdict(default_value_1)
cache_2 = defaultdict(default_value_2)


In [28]:
cache_1["a"], cache_1["b"], cache_1["c"]

__call__ Counter: 1
__call__ Counter: 2
__call__ Counter: 3


(None, None, None)

In [29]:
cache_2["a"], cache_2["b"], cache_2["c"]

__call__ Counter: 1
__call__ Counter: 2
__call__ Counter: 3


(0, 0, 0)

In [30]:
from time import perf_counter
from functools import wraps

def profiler(fn):
    counter = 0
    total_elapsed = 0
    avg_time = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal counter
        nonlocal total_elapsed
        nonlocal avg_time

        counter += 1
        start = perf_counter()
    
        result = fn(*args, **kwargs)
        
        end = perf_counter()
        total_elapsed += (end - start)
        avg_time = total_elapsed  / counter

        # one way to fix the problem - uncomment to check out
        # inner.counter = counter
        # inner.avg_time = avg_time
        
        return result

    # this won't work!
    inner.counter = counter
    inner.avg_time = avg_time
    return inner


In [31]:
from time import sleep
import random

random.seed(0)

@profiler
def func1():
    sleep(random.random())

func1(), func1(), func1()

(None, None, None)

In [32]:
func1.counter, func1.avg_time  # returns 0 every time !

(0, 0)

In [33]:
from time import perf_counter
from functools import wraps

def profiler(fn):
    _counter = 0
    _total_elapsed = 0
    _avg_time = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal _counter
        nonlocal _total_elapsed
        nonlocal _avg_time

        _counter += 1
        start = perf_counter()
        ###
        result = fn(*args, **kwargs)
        ###
        end = perf_counter()
        _total_elapsed += (end - start)
        _avg_time = _total_elapsed  / _counter
        return result

    def counter():
        return _counter

    def avg_time():
        return _avg_time


    inner.counter = counter  # these are functons
    inner.avg_time = avg_time
    return inner

In [34]:
from time import sleep
import random

random.seed(0)

@profiler
def func1():
    sleep(random.random())

func1(), func1(), func1()

(None, None, None)

In [35]:
func1.counter(), func1.avg_time()

(3, 0.677257146666913)

In [36]:
# Class based approach, class used as a decorator
class Profiler:

    def __init__(self, fn):
        self.counter = 0
        self.total_elapsed = 0
        self.fn = fn

    def __call__(self, *args, **kwargs):
        self.counter += 1
        start = perf_counter()
        ##
        result = self.fn(*args, **kwargs)
        ##
        end = perf_counter()
        self.total_elapsed +=  (end - start)
        return result

    @property
    def avg_time(self):
        return self.total_elapsed / self.counter


In [37]:
@Profiler  # func_1 = Profiler(func_1)
def func_1(a, b):
    sleep(random.random())
    return (a, b)

type(func_1), callable(func_1)

(__main__.Profiler, True)

In [38]:
func_1(1, 2), func_1(2, 3), func_1("a", "b")

((1, 2), (2, 3), ('a', 'b'))

In [39]:
func_1.counter, func_1.total_elapsed, func_1.avg_time

(3, 1.1862896910024574, 0.39542989700081915)